In [ ]:
import pandas as pd
import numpy as np
import math as m
import random as rand
import matplotlib.pyplot as plt
import seaborn as sns
import re
from datetime import datetime
from sklearn import linear_model as lm, metrics, ensemble as ens
from sklearn.model_selection import train_test_split, cross_val_score, RepeatedKFold
from sklearn.preprocessing import StandardScaler, normalize
from sklearn.svm import SVC
from sklearn.feature_selection import RFE, RFECV, SequentialFeatureSelector
import random
from collections import defaultdict
from itertools import islice

In [ ]:
#DEFINING A FUNCTION TO UPDATE COLUMN NAMES LATER
def lower_no_space(word): 
    
    word = re.sub(' ', '_', word) 
    
    word = re.sub(r'\'', '', word) 
    
    word = re.sub(r'\(', '', word)
    
    word = re.sub(r'\)', '', word)
    
    word = re.sub('\?', '', word)
    
    word = re.sub('/', '_', word)
    
    word = word.lower()
    
    return word

In [ ]:
#READ IN ORIGINAL CLINICAL DATA FOR LATER USE (CONVERTED TO .csv IN GOOGLE SHEETS)
df_clin = pd.read_csv("Homebase.csv", header = 1)

In [ ]:
#RENAMING COLUMNS
df_clin = df_clin.rename(mapper = lower_no_space, axis = 1) 
df_clin.rename(columns={'subject_sample_id':'sample_id'}, inplace=True)

In [ ]:
df_gen = pd.read_csv("full_gen.csv") #SEE GEN_DF_SETUP

In [ ]:
df_gen = df_gen[df_gen['chrom'] != 'Chromosome']
df_gen = df_gen[df_gen['gene_symbol'] != '#version 2.4'] #WEIRD OBS IN DATA
df_gen['chrom'] = df_gen['chrom'].apply(lambda x: '19' if 'gl000209' in str(x) else str(x)) #WEIRD OBS AGAIN
df_gen = df_gen.drop(columns = ['phred']) #USING ONLY RAWSCORE FOR NOW

In [ ]:
df_gen['pos'] = df_gen['pos'].astype('float')
df_gen = df_gen[~df_gen['pos'].isna()]


In [ ]:
# num_sections = m.ceil(np.max((df_gen['pos'].values.tolist()))/10000000)
#^^CAN USE TO DOUBLE CHECK THAT WE HAVE THE RIGHT NUMBER OF SECTIONS

df_gen['section'] = df_gen['pos'].apply(lambda x: m.ceil(x/1000000))
df_gen['section'] = df_gen['section'].astype('string')
df_gen['section'] = df_gen['section'].astype('string') + '_' + 'chrom' + '_' + df_gen['chrom']


In [ ]:
#CHECK GENERAL SHAPE
df_gen.head()

In [ ]:
# CREATING DICTIONARY TO COMPARE # OF PEOPLE W/ MUTATIONS FOR A GIVEN GENE IN EACH COHORT
genes_comp = {}

for chrom in ['11', '6', '17', '16', '5', '1', '2', '10', '7', '3', '19', '12', '4']: #chroms come from FI in classifier notebook
    for gene in set(df_gen[df_gen['chrom'] == chrom]['gene_symbol']):
        single_dict = {}
        single_dict['ss'] = len(set(df_gen[(df_gen['outcome'] == 1) & (df_gen['gene_symbol'] == gene)]['sample_id']))
        single_dict['mf'] = len(set(df_gen[(df_gen['outcome'] == 0) & (df_gen['gene_symbol'] == gene)]['sample_id']))
        single_dict['diff'] = single_dict['ss'] - single_dict['mf']
        genes_comp[gene] = single_dict

In [ ]:
# ORDER BY DIFF
genes_comp_ordered = {k: v for k, v in sorted(genes_comp.items(), key=lambda item: item[1]['diff'], reverse = True)}


In [ ]:
# TAKE A LOOK
genes_comp_ordered

In [ ]:
# PROPORTIONS WOULD BE BETTER
genes_comp_pct_ordered = genes_comp_ordered.copy()

for gene in genes_comp_pct_ordered.keys():
    genes_comp_pct_ordered[gene]['ss'] /= len(set(df_gen[df_gen['outcome'] == 1]['sample_id']))
    genes_comp_pct_ordered[gene]['mf'] /= len(set(df_gen[df_gen['outcome'] == 0]['sample_id']))
    genes_comp_pct_ordered[gene]['diff'] = genes_comp_pct_ordered[gene]['ss'] - genes_comp_pct_ordered[gene]['mf']

In [ ]:
# SHOULDN'T NEED TO RE-SORT, BUT JUST IN CASE

genes_comp_pct_ordered = {k: v for k, v in sorted(genes_comp_pct_ordered.items(), key=lambda item: item[1]['diff'], reverse = True)}
genes_comp_pct_ordered



In [ ]:
# MAKE A DF
impt_genes = pd.DataFrame(columns = ['gene', 'ss', 'mf', 'diff'])
for i in range(len(genes_comp_pct_ordered)):
    gene = list(genes_comp_pct_ordered.keys())[i]
    ss = genes_comp_pct_ordered[list(genes_comp_pct_ordered.keys())[i]]['ss']
    mf = genes_comp_pct_ordered[list(genes_comp_pct_ordered.keys())[i]]['mf']
    diff = genes_comp_pct_ordered[list(genes_comp_pct_ordered.keys())[i]]['diff']
    impt_genes.loc[i] = [gene, ss, mf, diff]

In [ ]:
# SAVE CSV
impt_genes.to_csv('gene_cohort_comparison.csv')

note:

chi-square of OBSCN mutation rates was significant at 10%:
https://www.socscistatistics.com/tests/chisquare/default2.aspx

but not sig. for Fisher Exact Test (probs more meaningful for now):
https://www.socscistatistics.com/tests/fisher/default2.aspx

so, potentially something there with a larger sample, but for now that's conjecture